# Narwhals DuckLake Example

An example notebook showing how to use the [Narwhals](https://narwhals-dev.github.io/narwhals/) dataframe library with DuckLake.

Install Narwhals and DuckDB:

In [ ]:
!pip install "narwhals[duckdb]"

Import the `duckdb` and `narwhals` packages:

In [2]:
import duckdb
import narwhals as nw

Connect to DuckLake:

In [3]:
con = duckdb.connect()
con.install_extension("ducklake")
con.sql("ATTACH 'ducklake:metadata.ducklake' AS my_ducklake")
con.sql("USE my_ducklake")

Create a table:

In [4]:
con.sql("""
CREATE TABLE IF NOT EXISTS penguins AS
    FROM 'http://blobs.duckdb.org/data/penguins.csv'
""")

Create a Narwhals LazyFrame from the table:

In [5]:
df = nw.from_native(con.table("penguins"))

Preview the LazyFrame:

In [6]:
df.head().to_native()

┌─────────┬───────────┬────────────────┬───────────────┬───────────────────┬─────────────┬─────────┬───────┐
│ species │  island   │ bill_length_mm │ bill_depth_mm │ flipper_length_mm │ body_mass_g │   sex   │ year  │
│ varchar │  varchar  │    varchar     │    varchar    │      varchar      │   varchar   │ varchar │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼─────────┼───────┤
│ Adelie  │ Torgersen │ 39.1           │ 18.7          │ 181               │ 3750        │ male    │  2007 │
│ Adelie  │ Torgersen │ 39.5           │ 17.4          │ 186               │ 3800        │ female  │  2007 │
│ Adelie  │ Torgersen │ 40.3           │ 18            │ 195               │ 3250        │ female  │  2007 │
│ Adelie  │ Torgersen │ NA             │ NA            │ NA                │ NA          │ NA      │  2007 │
│ Adelie  │ Torgersen │ 36.7           │ 19.3          │ 193               │ 3450        │ female  │  2007 │
└─────────┴────────

Get the count for each species:

In [7]:
df.group_by(nw.col("species")).agg(count=nw.len()).to_native()

┌───────────┬───────┐
│  species  │ count │
│  varchar  │ int64 │
├───────────┼───────┤
│ Adelie    │   152 │
│ Chinstrap │    68 │
│ Gentoo    │   124 │
└───────────┴───────┘

Close the connection:

In [8]:
con.close()